In [3]:
import pandas as pd

In [4]:
faoData = pd.read_csv("FAO database.csv", encoding='ISO-8859-1')
riceProduction = pd.read_csv("rice production across different countries from 1961 to 2021.csv", encoding='ISO-8859-1')

In [5]:

foodPrice = pd.read_csv("wfp_food_prices_database.csv", encoding='ISO-8859-1', low_memory=False)

In [6]:
faoData.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AF,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,...,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AF,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,...,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AF,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,...,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AF,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,...,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AF,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,...,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [7]:
riceProduction.head()

,ï»¿Area,Year,Unit,Value,Flag,Flag Description
0,Afghanistan,1961,tonnes,319000.0,A,Official figure
1,Afghanistan,1962,tonnes,319000.0,A,Official figure
2,Afghanistan,1963,tonnes,319000.0,A,Official figure
3,Afghanistan,1964,tonnes,380000.0,A,Official figure
4,Afghanistan,1965,tonnes,380000.0,A,Official figure


In [8]:
foodPrice.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [9]:

faoData = faoData[(faoData['Area'] == 'Madagascar') &
                  (faoData['Item'] == 'Rice (Milled Equivalent)')
                  # &
                  # (faoData['Element'] == 'Feed')
]

In [10]:

year_columns = [col for col in faoData.columns if col.startswith('Y')]

In [11]:

id_columns = ['Area', 'Item', 'Element', 'Unit']

In [12]:
faoData_long = pd.melt(faoData,
                       id_vars=id_columns,
                       value_vars=year_columns,
                       var_name='Year',
                       value_name='FAO_Value')

In [13]:

faoData_long['Year'] = faoData_long['Year'].str[1:].astype(int)

In [14]:
faoData_long

,Area,Item,Element,Unit,Year,FAO_Value
0,Madagascar,Rice (Milled Equivalent),Feed,1000 tonnes,1961,133.0
1,Madagascar,Rice (Milled Equivalent),Food,1000 tonnes,1961,641.0
2,Madagascar,Rice (Milled Equivalent),Feed,1000 tonnes,1962,147.0
3,Madagascar,Rice (Milled Equivalent),Food,1000 tonnes,1962,654.0
4,Madagascar,Rice (Milled Equivalent),Feed,1000 tonnes,1963,147.0
...,...,...,...,...,...,...
101,Madagascar,Rice (Milled Equivalent),Food,1000 tonnes,2011,2242.0
102,Madagascar,Rice (Milled Equivalent),Feed,1000 tonnes,2012,417.0
103,Madagascar,Rice (Milled Equivalent),Food,1000 tonnes,2012,2290.0
104,Madagascar,Rice (Milled Equivalent),Feed,1000 tonnes,2013,119.0


In [15]:
faoData_aggregated = faoData_long.pivot_table(
    index=['Area', 'Item', 'Year', 'Unit'],
    columns='Element',
    values='FAO_Value',
    aggfunc='sum',
    fill_value=0
).reset_index()

In [16]:
faoData_aggregated['FAO_Value'] = faoData_aggregated['Feed'] + faoData_aggregated['Food']


In [17]:
faoData_aggregated['Element'] = 'Feed'

In [18]:
faoData_final = faoData_aggregated.drop(columns=['Food', 'Feed'])


In [19]:
faoData_final

Element,Area,Item,Year,Unit,FAO_Value,Element
0,Madagascar,Rice (Milled Equivalent),1961,1000 tonnes,774.0,Feed
1,Madagascar,Rice (Milled Equivalent),1962,1000 tonnes,801.0,Feed
2,Madagascar,Rice (Milled Equivalent),1963,1000 tonnes,829.0,Feed
3,Madagascar,Rice (Milled Equivalent),1964,1000 tonnes,869.0,Feed
4,Madagascar,Rice (Milled Equivalent),1965,1000 tonnes,863.0,Feed
5,Madagascar,Rice (Milled Equivalent),1966,1000 tonnes,865.0,Feed
6,Madagascar,Rice (Milled Equivalent),1967,1000 tonnes,930.0,Feed
7,Madagascar,Rice (Milled Equivalent),1968,1000 tonnes,949.0,Feed
8,Madagascar,Rice (Milled Equivalent),1969,1000 tonnes,985.0,Feed
9,Madagascar,Rice (Milled Equivalent),1970,1000 tonnes,1021.0,Feed


In [20]:
faoData_final[faoData_final['Year'] == 2004]

Element,Area,Item,Year,Unit,FAO_Value,Element
43,Madagascar,Rice (Milled Equivalent),2004,1000 tonnes,1864.0,Feed


In [21]:
faoData_long[faoData_long['Year'] == 2004]

,Area,Item,Element,Unit,Year,FAO_Value
86,Madagascar,Rice (Milled Equivalent),Feed,1000 tonnes,2004,80.0
87,Madagascar,Rice (Milled Equivalent),Food,1000 tonnes,2004,1784.0


In [22]:
80+1784

1864

In [23]:
faoData.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,...,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
11758,MG,129,Madagascar,2805,Rice (Milled Equivalent),5521,Feed,1000 tonnes,-18.77,46.87,...,80.0,227.0,215.0,221.0,337.0,510.0,534.0,416.0,417,119
11759,MG,129,Madagascar,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,-18.77,46.87,...,1784.0,1852.0,1915.0,1983.0,2043.0,2124.0,2197.0,2242.0,2290,2350


In [24]:
riceProduction = riceProduction[riceProduction['ï»¿Area'] == 'Madagascar']

In [25]:
riceProduction['Area'] = riceProduction['ï»¿Area']

In [26]:

riceProduction.drop(riceProduction.columns[0], axis=1, inplace=True)

In [27]:
riceProduction.head()

,Year,Unit,Value,Flag,Flag Description,Area
3960,1961,tonnes,1465000.0,A,Official figure,Madagascar
3961,1962,tonnes,1552000.0,A,Official figure,Madagascar
3962,1963,tonnes,1559000.0,A,Official figure,Madagascar
3963,1964,tonnes,1648080.0,A,Official figure,Madagascar
3964,1965,tonnes,1589345.0,A,Official figure,Madagascar


In [28]:
foodPrice = foodPrice[
    (foodPrice['adm0_name'] == 'Madagascar') &
    (foodPrice['cm_name'].str.contains('Rice', case=False, na=False))
]


In [29]:
foodPrice.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
743242,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,1,2011,1240.0,NaN
743243,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,2,2011,1367.0,NaN
743244,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,3,2011,1550.0,NaN
743245,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,4,2011,1355.0,NaN
743246,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,5,2011,1355.0,NaN


In [30]:
mergedProdFao = pd.merge(riceProduction, faoData_final,
                       left_on=['Area', 'Year'],
                       right_on=['Area', 'Year'],
                       how='inner')

In [31]:
mergedProdFao

,Year,Unit_x,Value,Flag,Flag Description,Area,Item,Unit_y,FAO_Value,Element
0,1961,tonnes,1465000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,774.0,Feed
1,1962,tonnes,1552000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,801.0,Feed
2,1963,tonnes,1559000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,829.0,Feed
3,1964,tonnes,1648080.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,869.0,Feed
4,1965,tonnes,1589345.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,863.0,Feed
5,1966,tonnes,1602800.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,865.0,Feed
6,1967,tonnes,1706400.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,930.0,Feed
7,1968,tonnes,1796500.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,949.0,Feed
8,1969,tonnes,1844000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,985.0,Feed
9,1970,tonnes,1945900.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,1021.0,Feed


In [32]:
mergedProdFao = mergedProdFao.rename(columns={'Value': 'Product'})

In [33]:
mergedProdFao

,Year,Unit_x,Product,Flag,Flag Description,Area,Item,Unit_y,FAO_Value,Element
0,1961,tonnes,1465000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,774.0,Feed
1,1962,tonnes,1552000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,801.0,Feed
2,1963,tonnes,1559000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,829.0,Feed
3,1964,tonnes,1648080.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,869.0,Feed
4,1965,tonnes,1589345.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,863.0,Feed
5,1966,tonnes,1602800.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,865.0,Feed
6,1967,tonnes,1706400.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,930.0,Feed
7,1968,tonnes,1796500.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,949.0,Feed
8,1969,tonnes,1844000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,985.0,Feed
9,1970,tonnes,1945900.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,1021.0,Feed


In [34]:
mergedProdFao['FAO_Value'] =  mergedProdFao['FAO_Value'] * 1000

In [35]:
mergedProdFao

,Year,Unit_x,Product,Flag,Flag Description,Area,Item,Unit_y,FAO_Value,Element
0,1961,tonnes,1465000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,774000.0,Feed
1,1962,tonnes,1552000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,801000.0,Feed
2,1963,tonnes,1559000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,829000.0,Feed
3,1964,tonnes,1648080.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,869000.0,Feed
4,1965,tonnes,1589345.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,863000.0,Feed
5,1966,tonnes,1602800.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,865000.0,Feed
6,1967,tonnes,1706400.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,930000.0,Feed
7,1968,tonnes,1796500.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,949000.0,Feed
8,1969,tonnes,1844000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,985000.0,Feed
9,1970,tonnes,1945900.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,1021000.0,Feed


In [36]:
foodPrice

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
743242,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,1,2011,1240.0,NaN
743243,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,2,2011,1367.0,NaN
743244,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,3,2011,1550.0,NaN
743245,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,4,2011,1355.0,NaN
743246,150.0,Madagascar,1881,Atsinanana,750,Atsinanana,71,Rice (local) - Retail,0.0,MGA,15,Retail,5,KG,5,2011,1355.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751738,150.0,Madagascar,899999,Alaotra Mangoro,754,Alaotra Mangoro,203,Rice (paddy) - Producer,0.0,MGA,17,Producer,5,KG,5,2012,540.0,NaN
751739,150.0,Madagascar,899999,Alaotra Mangoro,754,Alaotra Mangoro,203,Rice (paddy) - Producer,0.0,MGA,17,Producer,5,KG,6,2012,462.0,NaN
751740,150.0,Madagascar,899999,Alaotra Mangoro,754,Alaotra Mangoro,203,Rice (paddy) - Producer,0.0,MGA,17,Producer,5,KG,7,2012,500.0,NaN
751741,150.0,Madagascar,899999,Alaotra Mangoro,754,Alaotra Mangoro,203,Rice (paddy) - Producer,0.0,MGA,17,Producer,5,KG,8,2012,544.0,NaN


In [37]:
final_merge = pd.merge(
    mergedProdFao,
    foodPrice,
    left_on=[ 'Year'],
    right_on=['mp_year'],
    how='inner'
)


In [38]:
final_merge

,Year,Unit_x,Product,Flag,Flag Description,Area,Item,Unit_y,FAO_Value,Element,...,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,2004,tonnes,3030000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,1864000.0,Feed,...,0.0,MGA,15,Retail,5,KG,1,2004,442.40,NaN
1,2004,tonnes,3030000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,1864000.0,Feed,...,0.0,MGA,15,Retail,5,KG,2,2004,485.10,NaN
2,2004,tonnes,3030000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,1864000.0,Feed,...,0.0,MGA,15,Retail,5,KG,3,2004,531.30,NaN
3,2004,tonnes,3030000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,1864000.0,Feed,...,0.0,MGA,15,Retail,5,KG,6,2004,731.50,NaN
4,2004,tonnes,3030000.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,1864000.0,Feed,...,0.0,MGA,15,Retail,5,KG,7,2004,878.50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,2013,tonnes,3610626.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,2469000.0,Feed,...,0.0,MGA,15,Retail,5,KG,8,2013,1083.00,NaN
1445,2013,tonnes,3610626.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,2469000.0,Feed,...,0.0,MGA,15,Retail,5,KG,9,2013,1147.25,NaN
1446,2013,tonnes,3610626.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,2469000.0,Feed,...,0.0,MGA,15,Retail,5,KG,10,2013,1193.00,NaN
1447,2013,tonnes,3610626.0,A,Official figure,Madagascar,Rice (Milled Equivalent),1000 tonnes,2469000.0,Feed,...,0.0,MGA,15,Retail,5,KG,11,2013,1270.50,NaN


In [39]:
final_merge['mp_price'] = final_merge['mp_price'] * 1000

In [40]:
final_merge.to_csv('final_merge.csv', index=False)